In [ ]:
#pip install lightgbm

In [ ]:
#!pip install pycaret[full]

# 모듈 불러오기

In [1]:
from pycaret.classification import *
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from tqdm import tqdm
import gc
import random
import lightgbm as lgb
import re
from sklearn.metrics import *
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
## 필요한 함수 정의


def make_datetime(x):
    # string 타입의 Time column을 datetime 타입으로 변경
    x     = str(x)
    year  = int(x[:4])
    month = int(x[4:6])
    day   = int(x[6:8])
    hour  = int(x[8:10])
    #mim  = int(x[10:12])
    #sec  = int(x[12:])
    return dt.datetime(year, month, day, hour)

def string2num(x):
    # (,)( )과 같은 불필요한 데이터 정제
    x = re.sub(r"[^0-9]+", '', str(x))
    if x =='':
        return 0
    else:
        return int(x)


PATH = 'data/'

## 파일 불러오기

In [3]:
train_err  = pd.read_csv(PATH+'train_err_data.csv')
display(train_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


In [4]:
train_quality  = pd.read_csv(PATH+'train_quality_data.csv')
display(train_quality.head())

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
1,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
2,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
3,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0
4,20201129090000,10000,05.15.2138,0.0,0,0.0,0,0,0,0,0,0,0,4,0,0


In [5]:
# 데이터 설명을 확인하면
# ueser_id가 10000부터 24999까지 총 15000개가 연속적으로 존재.
display(train_err.head())
train_user_id_max = 24999
train_user_id_min = 10000
train_user_number = 15000

,user_id,time,model_nm,fwver,errtype,errcode
0,10000,20201101025616,model_3,05.15.2138,15,1
1,10000,20201101030309,model_3,05.15.2138,12,1
2,10000,20201101030309,model_3,05.15.2138,11,1
3,10000,20201101050514,model_3,05.15.2138,16,1
4,10000,20201101050515,model_3,05.15.2138,4,0


### 테스트 데이터 불러오기

In [6]:
test_err  = pd.read_csv(PATH+'test_err_data.csv')
display(test_err.head())

,user_id,time,model_nm,fwver,errtype,errcode
0,30000,20201101030227,model_1,04.16.3553,31,1
1,30000,20201101030227,model_1,04.16.3553,33,2
2,30000,20201101030228,model_1,04.16.3553,15,1
3,30000,20201101030256,model_1,04.16.3553,22,1
4,30000,20201101030300,model_1,04.16.3553,11,1


In [7]:
test_quality  = pd.read_csv(PATH+'test_quality_data.csv')
display(test_quality.head())

,time,user_id,fwver,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
1,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
2,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
3,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0
4,20201128195000,30000,04.33.1261,0.0,0,0.0,0,0,0,0,5,0,0,2,0,0


In [8]:
# 데이터 설명을 확인하면
# test 데이터는 ueser_id가 30000부터 44998까지 총 14999개가 존재.
test_user_id_max = 44998
test_user_id_min = 30000
test_user_number = 14999

In [9]:
train_err['time_rank'] = train_err['time'].groupby(train_err['user_id']).rank(ascending=True)

In [10]:
print(np.sort(train_err.errtype.unique()))
# errtype이 1부터 42까지 29를 제외한 41개가 존재.

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 30 31 32 33 34 35 36 37 38 39 40 41 42]


In [11]:
print(np.sort(train_err.model_nm.unique()))

['model_0' 'model_1' 'model_2' 'model_3' 'model_4' 'model_5' 'model_6'
 'model_7' 'model_8']


In [12]:
train_err.model_nm.nunique()

9

### fwver 값 통일하기

In [13]:
print(np.sort(train_err.fwver.unique()))

['03.11.1141' '03.11.1149' '03.11.1167' '04.16.2641' '04.16.3345'
 '04.16.3439' '04.16.3553' '04.16.3569' '04.16.3571' '04.22.1442'
 '04.22.1656' '04.22.1666' '04.22.1684' '04.22.1750' '04.22.1778'
 '04.33.1095' '04.33.1125' '04.33.1149' '04.33.1171' '04.33.1185'
 '04.33.1261' '04.73.2237' '04.73.2571' '04.82.1684' '04.82.1730'
 '04.82.1778' '05.15.2090' '05.15.2092' '05.15.2114' '05.15.2120'
 '05.15.2122' '05.15.2138' '05.15.3104' '05.66.3237' '05.66.3571' '10'
 '8.5.3']


In [14]:
print(np.sort(test_err.fwver.unique()))

['03.11.1141' '03.11.1149' '03.11.1167' '04.16.3439' '04.16.3553'
 '04.16.3569' '04.16.3571' '04.22.1170' '04.22.1448' '04.22.1478'
 '04.22.1608' '04.22.1656' '04.22.1666' '04.22.1684' '04.22.1750'
 '04.22.1772' '04.22.1778' '04.33.1125' '04.33.1149' '04.33.1171'
 '04.33.1185' '04.33.1261' '04.73.2237' '04.73.2569' '04.73.2571'
 '04.73.2577' '04.82.1684' '04.82.1730' '04.82.1778' '05.15.2092'
 '05.15.2114' '05.15.2120' '05.15.2138' '05.15.3104' '05.66.3237'
 '05.66.3571' '10' '10.22.1770' '10.22.1780' '8.5.3']


In [15]:
train_err.fwver = train_err.fwver.apply(lambda x : x[0:5])
train_err.fwver.unique()

array(['05.15', '04.33', '04.22', '04.16', '05.66', '03.11', '04.82',
       '04.73', '10', '8.5.3'], dtype=object)

In [16]:
test_err.fwver = test_err.fwver.apply(lambda x : x[0:5])
test_err.fwver.unique()

array(['04.16', '04.33', '05.15', '04.22', '03.11', '04.82', '10',
       '05.66', '04.73', '8.5.3', '10.22'], dtype=object)

In [17]:
test_err.fwver = test_err.fwver.replace('10.22', '10')

In [18]:
len(np.sort(train_err.fwver.unique()))

10

In [19]:
len(np.sort(test_err.fwver.unique()))

10

### errcode 상위 40개 추려내기

In [20]:
pd.value_counts(train_err['errcode'].values, sort=True)

1                     8906967
0                     2599123
connection timeout    1835600
B-A8002                897863
80                     334018
                       ...   
5046                        1
3429                        1
16158                       1
5007                        1
51324                       1
Length: 2805, dtype: int64

In [21]:
pd.value_counts(test_err['errcode'].values, sort=True)

1                     8750154
0                     2565532
connection timeout    1890632
B-A8002                855747
80                     326179
                       ...   
5466                        1
5396                        1
6051                        1
72543                       1
15972                       1
Length: 2955, dtype: int64

In [22]:
errcode_list_41 = pd.value_counts(train_err['errcode'].values, sort=True).index[:41].tolist()
errcode_list_41

['1',
 '0',
 'connection timeout',
 'B-A8002',
 '80',
 '79',
 '14',
 'active',
 '2',
 '84',
 '85',
 'standby',
 'NFANDROID2',
 'connection fail to establish',
 '3',
 '90',
 '89',
 'S-61001',
 '95',
 '94',
 '4',
 '13',
 'Q-64002',
 '8.0',
 '6',
 '5',
 '78',
 '81',
 '86',
 'connectionterminated by local host',
 'UNKNOWN',
 '100',
 'terminate by peer user',
 '99',
 '83',
 'V-21008',
 'S-65002',
 '87',
 '88',
 'H-51042',
 '82']

In [23]:
# ERR_CODE TOP 41개를제외하고 나머지는 etc로 통일
def err_func(err_code):
    if(err_code in errcode_list_41):
        return err_code
    else:
        return 'etc' 

In [24]:
train_err['errcode'] = train_err['errcode'].apply(lambda x : err_func(x))

In [25]:
test_err['errcode'] = test_err['errcode'].apply(lambda x : err_func(x))

In [26]:
train_err['errcode'].unique()

array(['1', '0', '2', 'NFANDROID2', 'B-A8002', '14', '4', '13', '3',
       '8.0', 'standby', 'active', 'Q-64002', 'connection timeout',
       'S-61001', 'etc', 'terminate by peer user',
       'connection fail to establish', '80', '79', '81', '86', '84',
       'connectionterminated by local host', '78', 'UNKNOWN', '85', '90',
       '89', '88', 'V-21008', '83', '95', '94', '87', '82', 'S-65002',
       '6', '5', 'H-51042', '100', '99'], dtype=object)

In [27]:
len(train_err['errcode'].unique())

42

In [28]:
len(test_err['errcode'].unique())

42

## 라벨 달아주기

In [29]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()

In [30]:
train_err_label = train_err[['model_nm','fwver','errcode']].apply(le.fit_transform) 

In [31]:
train_err_model = pd.DataFrame()
train_err_model['model_nm'] = train_err_label['model_nm']
train_err_model['user_id'] = train_err['user_id']

In [32]:
train_err_fwver = pd.DataFrame()
train_err_fwver['fwver'] = train_err_label['fwver']
train_err_fwver['user_id'] = train_err['user_id']

In [33]:
train_err_code = pd.DataFrame()
train_err_code['errcode'] = train_err_label['errcode']
train_err_code['user_id'] = train_err['user_id']

In [34]:
test_err_label = test_err[['model_nm','fwver','errcode']].apply(le.fit_transform) 
test_err_model = pd.DataFrame()
test_err_model['model_nm'] = test_err_label['model_nm']
test_err_model['user_id'] = test_err['user_id']
test_err_fwver = pd.DataFrame()
test_err_fwver['fwver'] = test_err_label['fwver']
test_err_fwver['user_id'] = test_err['user_id']
test_err_code = pd.DataFrame()
test_err_code['errcode'] = test_err_label['errcode']
test_err_code['user_id'] = test_err['user_id']

### Quality 데이터 가공

#### 1. 퀄리티 데이터를 얼마나 조회했는가
#### 2. 퀄리티 데이터의 평균 정보 입력

In [200]:
train_quality = train_quality.fillna(0)
train_quality = train_quality.replace(np.nan,0)
#train_quality = train_quality.apply(pd.to_numeric)
train_quality.quality_5 = train_quality.quality_5.str.replace(',', '').astype('int64')
train_quality.quality_7 = train_quality.quality_7.str.replace(',', '').astype('int64')
train_quality.quality_8 = train_quality.quality_8.str.replace(',', '').astype('int64')
train_quality.quality_9 = train_quality.quality_9.str.replace(',', '').astype('int64')
train_quality.quality_10 = train_quality.quality_10.str.replace(',', '').astype('int64')

In [55]:
cols = train_quality.columns
cols = [x for x in cols if 'quality' in x]

In [57]:
cols

['quality_0',
 'quality_1',
 'quality_2',
 'quality_3',
 'quality_4',
 'quality_5',
 'quality_6',
 'quality_7',
 'quality_8',
 'quality_9',
 'quality_10',
 'quality_11',
 'quality_12']

In [157]:
tq = train_quality.groupby(['user_id','time']).nunique()-1
tq = tq.reset_index()
tq.drop(['fwver'],1,inplace=True)
tq

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,20201129090000,0,0,0,0,0,1,0,0,0,0,0,0,0
1,10000,20201130210000,0,0,0,0,0,1,0,0,0,0,0,0,0
2,10002,20201104110000,1,0,1,0,0,0,0,0,0,0,0,0,0
3,10002,20201106010000,0,0,0,0,0,1,0,0,0,0,0,0,0
4,10002,20201111010000,0,0,0,0,0,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68947,24993,20201130203000,1,1,1,0,0,2,1,0,0,0,0,1,0
68948,24995,20201128202000,1,1,1,0,0,2,1,0,0,0,0,1,0
68949,24995,20201129002000,1,1,1,0,0,2,1,0,0,0,0,1,0
68950,24997,20201108230000,1,1,1,0,0,2,1,0,0,0,0,1,0


In [162]:
#퀄리티 데이터가 없는 데이터프레임
df_no_q = pd.DataFrame(columns=tq.columns)
no_q_user_id = list(set(list(range(train_user_id_min, train_user_id_max+1))) - set(tq.user_id.unique()))
df_no_q['user_id'] = no_q_user_id
df_no_q = df_no_q.fillna(0)
df_no_q

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,10003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10007,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,10008,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10009,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6714,24991,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6715,24994,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6716,24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6717,24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [163]:
#퀄리티 데이터의 카운트
tq_count = tq.groupby('user_id').count()
tq_count = tq_count.reset_index()
#tq_count = np.asarray(tq_count['time'])
#tq_count = tq_count.reshape(-1,1)
tq_count

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,10002,8,8,8,8,8,8,8,8,8,8,8,8,8,8
2,10004,2,2,2,2,2,2,2,2,2,2,2,2,2,2
3,10005,2,2,2,2,2,2,2,2,2,2,2,2,2,2
4,10006,3,3,3,3,3,3,3,3,3,3,3,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8277,24992,3,3,3,3,3,3,3,3,3,3,3,3,3,3
8278,24993,2,2,2,2,2,2,2,2,2,2,2,2,2,2
8279,24995,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [166]:
df_tq_count = pd.concat([df_no_q,tq_count]) 
df_tq_count = df_tq_count.sort_values('user_id').reset_index()
df_tq_count.drop("index",1,inplace=True)
df_tq_count

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,2,2,2,2,2,2,2,2,2,2,2,2,2,2
1,10001,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10002,8,8,8,8,8,8,8,8,8,8,8,8,8,8
3,10003,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,10004,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,24995,2,2,2,2,2,2,2,2,2,2,2,2,2,2
14996,24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14997,24997,2,2,2,2,2,2,2,2,2,2,2,2,2,2
14998,24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [167]:
#Quality Data의 변화를 알아 보는 데이터
tq = tq.groupby('user_id')[cols].sum().reset_index()
tq

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0,0,0,0,0,2,0,0,0,0,0,0,0
1,10002,2,1,2,0,0,12,3,0,0,0,0,1,0
2,10004,1,1,1,0,0,4,2,0,0,0,0,1,0
3,10005,1,1,1,0,0,4,2,0,0,0,0,1,0
4,10006,0,0,0,0,0,3,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,-2,1,1,0,0,3,1,0,0,0,0,1,0
8277,24992,2,2,2,0,0,6,2,0,0,0,0,2,0
8278,24993,2,2,2,0,0,4,2,0,0,0,0,2,0
8279,24995,2,2,2,0,0,4,2,0,0,0,0,2,0


In [307]:
df_tq_change = pd.concat([tq,df_no_q]) 
df_tq_change = df_tq_change.sort_values('user_id').reset_index()
df_tq_change.drop("index",1,inplace=True)
df_tq_change.drop("user_id",1,inplace=True)
df_tq_change.drop("time",1,inplace=True)
df_tq_change

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0,0,0,0,0,2,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,1,2,0,0,12,3,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,1,1,0,0,4,2,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,2,2,2,0,0,4,2,0,0,0,0,2,0
14996,0,0,0,0,0,0,0,0,0,0,0,0,0
14997,1,1,1,0,0,6,1,0,0,0,0,1,0
14998,0,0,0,0,0,0,0,0,0,0,0,0,0


In [205]:
#Quality Data의 합을 알아 보는 데이터
tq_sum = train_quality.groupby(['user_id','time']).sum()
tq_sum

quality_0  quality_1  quality_2  quality_3  quality_4  \
user_id time                                                                    
10000   20201129090000        0.0          0        0.0          0          0   
        20201130210000        0.0          0        0.0          0          0   
10002   20201104110000        2.0          0        1.0          0          0   
        20201106010000        0.0          0        0.0          0          0   
        20201111010000        0.0          0        0.0          0          0   
...                           ...        ...        ...        ...        ...   
24993   20201130203000       -3.0         -3       -3.0          0          0   
24995   20201128202000       -8.0         -8       -8.0          0          0   
        20201129002000       -2.0         -2       -2.0          0          0   
24997   20201108230000       -4.0         -4       -4.0          0          0   
        20201124033000        0.0          0        0.0          0          0   

                        quality_5  quality_6  quality_7  quality_8  quality_9  \
user_id time                                                                    
10000   20201129090000          4          0          0          0        0.0   
        20201130210000          8          0          0          0        0.0   
10002   20201104110000          0          0          0          0       12.0   
        20201106010000          2          0          0          0        0.0   
        20201111010000          4          2         24          0        0.0   
...                           ...        ...        ...        ...        ...   
24993   20201130203000          2         -3          0          0        0.0   
24995   20201128202000         -6         -8          0          0        0.0   
        20201129002000          1         -2          0          0        0.0   
24997   20201108230000         -2         -4          0          0        0.0   
        20201124033000         17          0          0          0        0.0   

                        quality_10  quality_11  quality_12  
user_id time                                                
10000   20201129090000        48.0           0           0  
        20201130210000        96.0           0           0  
10002   20201104110000         0.0           0           0  
        20201106010000        24.0           0           0  
        20201111010000        48.0           0           0  
...                            ...         ...         ...  
24993   20201130203000        60.0          -3           0  
24995   20201128202000        24.0          -8           0  
        20201129002000        36.0          -2           0  
24997   20201108230000        24.0          -4           0  
        20201124033000       204.0           0           0  

[68952 rows x 13 columns]

In [206]:
tq_sum = tq_sum.groupby('user_id')[cols].sum().reset_index()
tq_sum

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.0,0,0.0,0,0,12,0,0,0,0.0,144.0,0,0
1,10002,0.0,-2,-1.0,0,0,29,44,552,0,12.0,372.0,-2,0
2,10004,-2.0,-2,-2.0,0,0,2,85,1044,0,0.0,48.0,-2,0
3,10005,-10.0,-10,-10.0,0,0,0,26,432,0,0.0,120.0,-10,0
4,10006,0.0,0,0.0,0,0,12,4,48,0,0.0,144.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,0.0,-8,-8.0,0,0,-6,-8,0,0,0.0,24.0,-8,0
8277,24992,-12.0,-12,-12.0,0,0,7,-12,0,0,0.0,228.0,-12,0
8278,24993,-7.0,-7,-7.0,0,0,1,-7,0,0,0.0,96.0,-7,0
8279,24995,-10.0,-10,-10.0,0,0,-5,-10,0,0,0.0,60.0,-10,0


In [208]:
df_tq_sum = pd.concat([tq_sum,df_no_q]) 
df_tq_sum = df_tq_sum.sort_values('user_id').reset_index()
df_tq_sum.drop("index",1,inplace=True)
df_tq_sum.drop("user_id",1,inplace=True)
df_tq_sum.drop('time',1,inplace=True)
df_tq_sum

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0,0.0,0,0,12,0,0,0,0.0,144.0,0,0
1,0.0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0
2,0.0,-2,-1.0,0,0,29,44,552,0,12.0,372.0,-2,0
3,0.0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0
4,-2.0,-2,-2.0,0,0,2,85,1044,0,0.0,48.0,-2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-10.0,-10,-10.0,0,0,-5,-10,0,0,0.0,60.0,-10,0
14996,0.0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0
14997,-4.0,-4,-4.0,0,0,15,-4,0,0,0.0,228.0,-4,0
14998,0.0,0,0.0,0,0,0,0,0,0,0.0,0.0,0,0


In [201]:
#Quality Data의 평균을 알아 보는 데이터
tq_mean = train_quality.groupby(['user_id','time']).mean()
tq_mean = tq_mean.reset_index()
tq_mean

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,20201129090000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,4.0,0.000000,0.0
1,10000,20201130210000,0.000000,0.000000,0.000000,0.0,0.0,0.666667,0.000000,0.0,0.0,0.0,8.0,0.000000,0.0
2,10002,20201104110000,0.166667,0.000000,0.083333,0.0,0.0,0.000000,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0
3,10002,20201106010000,0.000000,0.000000,0.000000,0.0,0.0,0.166667,0.000000,0.0,0.0,0.0,2.0,0.000000,0.0
4,10002,20201111010000,0.000000,0.000000,0.000000,0.0,0.0,0.333333,0.166667,2.0,0.0,0.0,4.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68947,24993,20201130203000,-0.250000,-0.250000,-0.250000,0.0,0.0,0.166667,-0.250000,0.0,0.0,0.0,5.0,-0.250000,0.0
68948,24995,20201128202000,-0.666667,-0.666667,-0.666667,0.0,0.0,-0.500000,-0.666667,0.0,0.0,0.0,2.0,-0.666667,0.0
68949,24995,20201129002000,-0.166667,-0.166667,-0.166667,0.0,0.0,0.083333,-0.166667,0.0,0.0,0.0,3.0,-0.166667,0.0
68950,24997,20201108230000,-0.333333,-0.333333,-0.333333,0.0,0.0,-0.166667,-0.333333,0.0,0.0,0.0,2.0,-0.333333,0.0


In [209]:
tq_mean = tq_mean.groupby('user_id')[cols].sum().reset_index()
tq_mean

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,10000,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,12.0,0.000000,0.0
1,10002,0.000000,-0.166667,-0.083333,0.0,0.0,2.416667,3.666667,46.0,0.0,1.0,31.0,-0.166667,0.0
2,10004,-0.166667,-0.166667,-0.166667,0.0,0.0,0.166667,7.083333,87.0,0.0,0.0,4.0,-0.166667,0.0
3,10005,-0.833333,-0.833333,-0.833333,0.0,0.0,0.000000,2.166667,36.0,0.0,0.0,10.0,-0.833333,0.0
4,10006,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.333333,4.0,0.0,0.0,12.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8276,24990,0.000000,-0.666667,-0.666667,0.0,0.0,-0.500000,-0.666667,0.0,0.0,0.0,2.0,-0.666667,0.0
8277,24992,-1.000000,-1.000000,-1.000000,0.0,0.0,0.583333,-1.000000,0.0,0.0,0.0,19.0,-1.000000,0.0
8278,24993,-0.583333,-0.583333,-0.583333,0.0,0.0,0.083333,-0.583333,0.0,0.0,0.0,8.0,-0.583333,0.0
8279,24995,-0.833333,-0.833333,-0.833333,0.0,0.0,-0.416667,-0.833333,0.0,0.0,0.0,5.0,-0.833333,0.0


In [210]:
df_tq_mean = pd.concat([tq_mean,df_no_q]) 
df_tq_mean = df_tq_mean.sort_values('user_id').reset_index()
df_tq_mean.drop("index",1,inplace=True)
df_tq_mean.drop("user_id",1,inplace=True)
df_tq_mean.drop('time',1,inplace=True)
df_tq_mean

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.000000,0.000000,0.000000,0.0,0.0,1.000000,0.000000,0.0,0.0,0.0,12.0,0.000000,0.0
1,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.000000,-0.166667,-0.083333,0.0,0.0,2.416667,3.666667,46.0,0.0,1.0,31.0,-0.166667,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
4,-0.166667,-0.166667,-0.166667,0.0,0.0,0.166667,7.083333,87.0,0.0,0.0,4.0,-0.166667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14995,-0.833333,-0.833333,-0.833333,0.0,0.0,-0.416667,-0.833333,0.0,0.0,0.0,5.0,-0.833333,0.0
14996,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
14997,-0.333333,-0.333333,-0.333333,0.0,0.0,1.250000,-0.333333,0.0,0.0,0.0,19.0,-0.333333,0.0
14998,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0


In [310]:
quality_error = np.hstack([np.asarray(df_tq_change),np.asarray(df_tq_count['time']).reshape(-1,1),np.asarray(df_tq_sum),np.asarray(df_tq_mean)])
quality_error

array([[ 0.        ,  0.        ,  0.        , ..., 12.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 2.        ,  1.        ,  2.        , ..., 31.        ,
        -0.16666667,  0.        ],
       ...,
       [ 1.        ,  1.        ,  1.        , ..., 19.        ,
        -0.33333333,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])

In [311]:
quality_error.shape

(15000, 40)

### errtype 데이터 생성

In [220]:
# user_id와 errtype만을 사용하여 데이터 셋 생성
# 모든 일자에 대해서 errtype별 발생 건수를 count
# pandas의 groupby를 활용할 경우 큰 연산 자원이 필요.
# numpy로 placeholder를 만들어 구현함.
id_type_error = train_err[['user_id','errtype']].values
type_error = np.zeros((train_user_number,42)) # 42 + 35 + 8 + 2(errcode)

for person_idx, err in tqdm(id_type_error):
    # person_idx - train_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    type_error[person_idx - train_user_id_min,err - 1] += 1
type_error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:46<00:00, 355021.18it/s]


(15000, 42)

### model_nm 데이터 생성

In [221]:
id_model_error = train_err_model[['user_id','model_nm']].values
model_error = np.zeros((train_user_number,9)) # 모델 카테고리 9개

for person_idx, err in tqdm(id_model_error):
    model_error[person_idx - train_user_id_min,err - 1] += 1
model_error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:48<00:00, 341871.31it/s]


(15000, 9)

### fwver 데이터 생성

In [222]:
id_fwver_error = train_err_fwver[['user_id','fwver']].values
fwver_error = np.zeros((train_user_number,10)) # 펌웨어 카테고리 10개

for person_idx, err in tqdm(id_fwver_error):
    fwver_error[person_idx - train_user_id_min,err - 1] += 1
fwver_error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:46<00:00, 357953.00it/s]


(15000, 10)

### errcode 데이터 생성

In [223]:
id_code_error = train_err_code[['user_id','errcode']].values
code_error = np.zeros((train_user_number,42)) # code 카테고리 42개

for person_idx, err in tqdm(id_code_error):
    code_error[person_idx - train_user_id_min,err - 1] += 1
code_error.shape

100%|██████████████████████████████████████████████████████████████████| 16554663/16554663 [00:46<00:00, 354228.86it/s]


(15000, 42)

In [231]:
type_error

array([[  0.,   0.,   8., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ..., 113.,  56.,   1.],
       [  0.,   0.,   2., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ...,  58.,   8.,   5.],
       [  0.,   0.,   0., ...,   6.,   0.,   0.],
       [  0.,   0.,   4., ...,   0.,   0.,   0.]])

In [232]:
model_error

array([[   0.,    0.,  317., ...,    0.,    0.,    0.],
       [   0., 2365.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,  306., ...,    0.,    0.,    0.],
       ...,
       [   0.,    0.,    0., ...,    0.,    0.,  826.],
       [   0.,    0.,    0., ...,    0.,    0.,  155.],
       [   0.,    0.,  570., ...,    0.,    0.,    0.]])

In [233]:
fwver_error

array([[   0.,    0.,    0., ...,    0.,    0.,    0.],
       [   0.,    0., 2365., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.],
       ...,
       [   0.,  826.,    0., ...,    0.,    0.,    0.],
       [   0.,  155.,    0., ...,    0.,    0.,    0.],
       [   0.,    0.,    0., ...,    0.,    0.,    0.]])

In [234]:
code_error

array([[2.120e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.040e+02],
       [1.274e+03, 0.000e+00, 1.000e+00, ..., 6.250e+02, 0.000e+00,
        1.830e+02],
       [1.720e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.320e+02],
       ...,
       [5.980e+02, 0.000e+00, 1.000e+00, ..., 1.100e+01, 0.000e+00,
        1.460e+02],
       [1.250e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.300e+01],
       [3.670e+02, 0.000e+00, 0.000e+00, ..., 0.000e+00, 0.000e+00,
        1.920e+02]])

In [228]:
train_prob = pd.read_csv(PATH+'train_problem_data.csv')
problem = np.zeros(15000)
# error와 동일한 방법으로 person_idx - 10000 위치에 
# person_idx의 problem이 한 번이라도 발생했다면 1
# 없다면 0
problem[train_prob.user_id.unique()-10000] = 1 
problem.shape

(15000,)

In [229]:
problem

array([0., 1., 0., ..., 1., 1., 0.])

In [314]:
# 변수 이름 변경
# error  -> train_x
# problem-> train_y

train_x = np.hstack([type_error, model_error,fwver_error,code_error,quality_error])#error
train_y = problem
print(train_x.shape)
print(train_y.shape)

(15000, 143)
(15000,)


In [315]:
train = pd.DataFrame(data=train_x)
train['problem'] = problem

In [316]:
clf = setup(data = train, target = 'problem') 

,Description,Value
0,session_id,2180
1,Target,problem
2,Target Type,Binary
3,Label Encoded,"0.0: 0, 1.0: 1"
4,Original Data,"(15000, 144)"
5,Missing Values,False
6,Numeric Features,140
7,Categorical Features,3
8,Ordinal Features,False
9,High Cardinality Features,False


In [317]:
best = compare_models(sort = 'AUC', n_select = 5)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
catboost,CatBoost Classifier,0.7963,0.8178,0.5209,0.7951,0.6291,0.4971,0.5184,11.2750
gbc,Gradient Boosting Classifier,0.7877,0.8121,0.4885,0.7926,0.6040,0.4701,0.4960,1.5390
lightgbm,Light Gradient Boosting Machine,0.7937,0.8113,0.5318,0.7767,0.6310,0.4948,0.5121,0.2680
rf,Random Forest Classifier,0.7846,0.8068,0.4928,0.7769,0.6028,0.4652,0.4880,0.8310
et,Extra Trees Classifier,0.7845,0.8024,0.5080,0.7644,0.6099,0.4694,0.4883,0.7880
xgboost,Extreme Gradient Boosting,0.7852,0.7983,0.5384,0.7441,0.6245,0.4796,0.4920,1.2520
ada,Ada Boost Classifier,0.7741,0.7960,0.4802,0.7498,0.5850,0.4400,0.4608,0.3490
lda,Linear Discriminant Analysis,0.7491,0.7481,0.3412,0.7800,0.4743,0.3414,0.3918,0.0930
qda,Quadratic Discriminant Analysis,0.7216,0.7264,0.3351,0.6653,0.4381,0.2821,0.3140,0.0550
nb,Naive Bayes,0.6793,0.6734,0.1276,0.5786,0.2087,0.1011,0.1469,0.0340


In [318]:
blended = blend_models(estimator_list = best, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7986,0.8153,0.5029,0.8220,0.6240,0.4971,0.5251
1,0.7890,0.8153,0.5057,0.7827,0.6144,0.4783,0.4999
2,0.7914,0.8216,0.5215,0.7778,0.6244,0.4877,0.5063
3,0.7924,0.8112,0.5237,0.7783,0.6261,0.4899,0.5083
4,0.7942,0.8200,0.4993,0.8074,0.6170,0.4868,0.5131
Mean,0.7931,0.8167,0.5106,0.7936,0.6212,0.4880,0.5105
SD,0.0032,0.0037,0.0100,0.0179,0.0046,0.0060,0.0084


In [319]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.7992,0.8282,0.5132,0.8220,0.6319,0.5038,0.5302


In [320]:
#final_model = best
final_model = finalize_model(blended)

### TEST 데이터 시작

In [242]:
test_quality.dtypes

time            int64
user_id         int64
fwver          object
quality_0     float64
quality_1      object
quality_2     float64
quality_3       int64
quality_4       int64
quality_5      object
quality_6       int64
quality_7      object
quality_8      object
quality_9      object
quality_10     object
quality_11      int64
quality_12      int64
dtype: object

In [248]:
test_quality = test_quality.fillna(0)
test_quality = test_quality.replace(np.nan,0)
train_quality = train_quality.apply(pd.to_numeric)
test_quality.quality_1 = test_quality.quality_1.str.replace(',', '').astype('float32')
test_quality.quality_5 = test_quality.quality_5.str.replace(',', '').astype('float32')
test_quality.quality_7 = test_quality.quality_7.str.replace(',', '').astype('int64')
test_quality.quality_8 = test_quality.quality_8.str.replace(',', '').astype('float32')
test_quality.quality_9 = test_quality.quality_9.str.replace(',', '').astype('float32')
test_quality.quality_10 = test_quality.quality_10.str.replace(',', '').astype('int64')

In [266]:
teq = test_quality.groupby(['user_id','time']).nunique()-1
teq = teq.reset_index()
teq.drop('fwver',1,inplace=True)
teq

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,20201128195000,0,0,0,0,0,1,1,0,0,0,0,0,0
1,30001,20201102202000,1,0,1,0,0,2,1,0,0,0,0,1,0
2,30001,20201104202000,0,0,0,0,0,2,0,0,0,0,0,0,0
3,30001,20201112040000,0,0,0,0,0,3,0,0,0,0,0,0,0
4,30002,20201101000000,0,0,0,0,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62257,44996,20201125011000,0,0,0,0,0,2,0,0,0,-1,0,0,0
62258,44996,20201126011000,1,0,1,0,0,3,1,0,0,-1,0,1,0
62259,44996,20201127111000,1,0,1,0,0,2,1,0,0,-1,0,1,0
62260,44997,20201102214000,0,0,0,0,0,2,0,0,0,-1,0,0,0


In [267]:
#퀄리티 데이터가 없는 데이터프레임
no_teq_user_id = list(set(list(range(test_user_id_min, test_user_id_max+1))) - set(teq.user_id.unique()))
df_no_tq = pd.DataFrame(columns=teq.columns)
df_no_tq['user_id'] = no_teq_user_id
df_no_tq = df_no_tq.fillna(0)
df_no_tq

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,32768,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,32773,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,32776,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,32780,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,32782,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6726,32759,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6727,32760,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6728,32762,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6729,32763,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [287]:
#퀄리티 데이터의 카운트
teq_count = teq.groupby('user_id').count()
teq_count = teq_count.reset_index()
teq_count

,index,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0,30000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,1,30001,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,2,30002,10,10,10,10,10,10,10,10,10,10,10,10,10,10
3,3,30003,5,5,5,5,5,5,5,5,5,5,5,5,5,5
4,4,30004,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,8263,44990,3,3,3,3,3,3,3,3,3,3,3,3,3,3
8264,8264,44993,4,4,4,4,4,4,4,4,4,4,4,4,4,4
8265,8265,44994,1,1,1,1,1,1,1,1,1,1,1,1,1,1
8266,8266,44996,9,9,9,9,9,9,9,9,9,9,9,9,9,9


In [288]:
df_teq_count = pd.concat([df_no_tq,teq_count]) 
df_teq_count = df_teq_count.sort_values('user_id').reset_index()
df_teq_count.drop("index",1,inplace=True)
df_teq_count

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,1,1,1,1,1,1,1,1,1,1,1,1,1,1
1,30001,3,3,3,3,3,3,3,3,3,3,3,3,3,3
2,30002,10,10,10,10,10,10,10,10,10,10,10,10,10,10
3,30003,5,5,5,5,5,5,5,5,5,5,5,5,5,5
4,30004,2,2,2,2,2,2,2,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,44994,1,1,1,1,1,1,1,1,1,1,1,1,1,1
14995,44995,0,0,0,0,0,0,0,0,0,0,0,0,0,0
14996,44996,9,9,9,9,9,9,9,9,9,9,9,9,9,9
14997,44997,2,2,2,2,2,2,2,2,2,2,2,2,2,2


In [269]:
#Quality Data의 변화를 알아 보는 데이터
teq = teq.groupby('user_id')[cols].sum().reset_index()
teq

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0,0,0,0,0,1,1,0,0,0,0,0,0
1,30001,1,0,1,0,0,7,1,0,0,0,0,1,0
2,30002,3,0,3,0,0,16,3,0,0,0,0,3,0
3,30003,0,0,0,0,0,5,5,0,0,0,0,0,0
4,30004,1,0,1,0,0,2,3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,44990,2,0,2,0,0,7,2,0,0,-3,0,2,0
8264,44993,4,0,4,0,0,6,5,0,0,-4,0,1,0
8265,44994,1,0,1,0,0,1,1,0,0,-1,0,1,0
8266,44996,2,0,2,0,0,16,2,0,0,-9,0,2,0


In [304]:
df_teq_change = pd.concat([teq,df_no_tq]) 
df_teq_change = df_teq_change.sort_values('user_id').reset_index()
df_teq_change.drop("index",1,inplace=True)
df_teq_change.drop("user_id",1,inplace=True)
df_teq_change.drop("time",1,inplace=True)
df_teq_change

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0,0,0,0,0,1,1,0,0,0,0,0,0
1,1,0,1,0,0,7,1,0,0,0,0,1,0
2,3,0,3,0,0,16,3,0,0,0,0,3,0
3,0,0,0,0,0,5,5,0,0,0,0,0,0
4,1,0,1,0,0,2,3,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,1,0,1,0,0,1,1,0,0,-1,0,1,0
14995,0,0,0,0,0,0,0,0,0,0,0,0,0
14996,2,0,2,0,0,16,2,0,0,-9,0,2,0
14997,0,0,0,0,0,4,0,0,0,-2,0,0,0


In [280]:
#Quality Data의 합을 알아 보는 데이터
teq_sum = test_quality.groupby(['user_id','time']).sum()
teq_sum

quality_0  quality_1  quality_2  quality_3  quality_4  \
user_id time                                                                    
30000   20201128195000        0.0        0.0        0.0          0          0   
30001   20201102202000       -9.0        0.0       -9.0          0          0   
        20201104202000        0.0        0.0        0.0          0          0   
        20201112040000        0.0        0.0        0.0          0          0   
30002   20201101000000        0.0        0.0        0.0          0          0   
...                           ...        ...        ...        ...        ...   
44996   20201125011000        0.0        0.0        0.0          0          0   
        20201126011000       -3.0        0.0       -3.0          0          0   
        20201127111000       -2.0        0.0       -2.0          0          0   
44997   20201102214000        0.0        0.0        0.0          0          0   
        20201125180000        0.0        0.0        0.0          0          0   

                        quality_5  quality_6  quality_7  quality_8  quality_9  \
user_id time                                                                    
30000   20201128195000        2.0          5         60        0.0        0.0   
30001   20201102202000       -6.0         -9          0        0.0        0.0   
        20201104202000        4.0          0          0        0.0        0.0   
        20201112040000     2556.0          0          0        0.0        0.0   
30002   20201101000000        6.0          0          0        0.0        0.0   
...                           ...        ...        ...        ...        ...   
44996   20201125011000        9.0          0          0        0.0        0.0   
        20201126011000        2.0         -3          0        0.0        0.0   
        20201127111000        0.0         -2          0        0.0        0.0   
44997   20201102214000       10.0          0          0        0.0        0.0   
        20201125180000        3.0          0          0        0.0        0.0   

                        quality_10  quality_11  quality_12  
user_id time                                                
30000   20201128195000          24           0           0  
30001   20201102202000          36          -9           0  
        20201104202000          48           0           0  
        20201112040000       30672           0           0  
30002   20201101000000          72           0           0  
...                            ...         ...         ...  
44996   20201125011000         108           0           0  
        20201126011000          60          -3           0  
        20201127111000          24          -2           0  
44997   20201102214000         120           0           0  
        20201125180000          36           0           0  

[62262 rows x 13 columns]

In [281]:
teq_sum = teq_sum.groupby('user_id')[cols].sum().reset_index()
teq_sum

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0.0,0.0,0.0,0,0,2.0,5,60,0.0,0.0,24,0,0
1,30001,-9.0,0.0,-9.0,0,0,2554.0,-9,0,0.0,0.0,30756,-9,0
2,30002,-15.0,0.0,-15.0,0,0,118.0,-15,0,0.0,0.0,1596,-15,0
3,30003,0.0,0.0,0.0,0,0,12.0,220,2640,0.0,0.0,144,0,0
4,30004,-6.0,0.0,-6.0,0,0,-4.0,27,396,0.0,0.0,24,-6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,44990,-4.0,0.0,-4.0,0,0,46.0,-4,0,0.0,0.0,600,-4,0
8264,44993,7.0,0.0,7.0,0,0,12.0,107,1308,0.0,0.0,168,-2,0
8265,44994,-11.0,0.0,-11.0,0,0,-9.0,-11,0,0.0,0.0,24,-11,0
8266,44996,-5.0,0.0,-5.0,0,0,32.0,-5,0,0.0,0.0,444,-5,0


In [282]:
df_teq_sum = pd.concat([teq_sum,df_no_tq]) 
df_teq_sum = df_teq_sum.sort_values('user_id').reset_index()
df_teq_sum.drop("index",1,inplace=True)
df_teq_sum.drop("user_id",1,inplace=True)
df_teq_sum.drop('time',1,inplace=True)
df_teq_sum

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.0,0.0,0.0,0,0,2.0,5,60,0.0,0.0,24,0,0
1,-9.0,0.0,-9.0,0,0,2554.0,-9,0,0.0,0.0,30756,-9,0
2,-15.0,0.0,-15.0,0,0,118.0,-15,0,0.0,0.0,1596,-15,0
3,0.0,0.0,0.0,0,0,12.0,220,2640,0.0,0.0,144,0,0
4,-6.0,0.0,-6.0,0,0,-4.0,27,396,0.0,0.0,24,-6,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,-11.0,0.0,-11.0,0,0,-9.0,-11,0,0.0,0.0,24,-11,0
14995,0.0,0.0,0.0,0,0,0.0,0,0,0.0,0.0,0,0,0
14996,-5.0,0.0,-5.0,0,0,32.0,-5,0,0.0,0.0,444,-5,0
14997,0.0,0.0,0.0,0,0,13.0,0,0,0.0,0.0,156,0,0


In [283]:
#Quality Data의 평균을 알아 보는 데이터
teq_mean = test_quality.groupby(['user_id','time']).mean()
teq_mean = teq_mean.reset_index()
teq_mean

,user_id,time,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,20201128195000,0.000000,0.0,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0
1,30001,20201102202000,-0.750000,0.0,-0.750000,0.0,0.0,-0.500000,-0.750000,0.0,0.0,0.0,3.0,-0.750000,0.0
2,30001,20201104202000,0.000000,0.0,0.000000,0.0,0.0,0.333333,0.000000,0.0,0.0,0.0,4.0,0.000000,0.0
3,30001,20201112040000,0.000000,0.0,0.000000,0.0,0.0,213.000000,0.000000,0.0,0.0,0.0,2556.0,0.000000,0.0
4,30002,20201101000000,0.000000,0.0,0.000000,0.0,0.0,0.500000,0.000000,0.0,0.0,0.0,6.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62257,44996,20201125011000,0.000000,0.0,0.000000,0.0,0.0,0.750000,0.000000,0.0,0.0,NaN,9.0,0.000000,0.0
62258,44996,20201126011000,-0.250000,0.0,-0.250000,0.0,0.0,0.166667,-0.250000,0.0,0.0,NaN,5.0,-0.250000,0.0
62259,44996,20201127111000,-0.166667,0.0,-0.166667,0.0,0.0,0.000000,-0.166667,0.0,0.0,NaN,2.0,-0.166667,0.0
62260,44997,20201102214000,0.000000,0.0,0.000000,0.0,0.0,0.833333,0.000000,0.0,0.0,NaN,10.0,0.000000,0.0


In [284]:
teq_mean = teq_mean.groupby('user_id')[cols].sum().reset_index()
teq_mean

,user_id,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,30000,0.000000,0.0,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0
1,30001,-0.750000,0.0,-0.750000,0.0,0.0,212.833328,-0.750000,0.0,0.0,0.0,2563.0,-0.750000,0.0
2,30002,-1.250000,0.0,-1.250000,0.0,0.0,9.833333,-1.250000,0.0,0.0,0.0,133.0,-1.250000,0.0
3,30003,0.000000,0.0,0.000000,0.0,0.0,1.000000,18.333333,220.0,0.0,0.0,12.0,0.000000,0.0
4,30004,-0.500000,0.0,-0.500000,0.0,0.0,-0.333333,2.250000,33.0,0.0,0.0,2.0,-0.500000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8263,44990,-0.333333,0.0,-0.333333,0.0,0.0,3.833333,-0.333333,0.0,0.0,0.0,50.0,-0.333333,0.0
8264,44993,0.583333,0.0,0.583333,0.0,0.0,1.000000,8.916667,109.0,0.0,0.0,14.0,-0.166667,0.0
8265,44994,-0.916667,0.0,-0.916667,0.0,0.0,-0.750000,-0.916667,0.0,0.0,0.0,2.0,-0.916667,0.0
8266,44996,-0.416667,0.0,-0.416667,0.0,0.0,2.666667,-0.416667,0.0,0.0,0.0,37.0,-0.416667,0.0


In [285]:
df_teq_mean = pd.concat([teq_mean,df_no_tq]) 
df_teq_mean = df_teq_mean.sort_values('user_id').reset_index()
df_teq_mean.drop("index",1,inplace=True)
df_teq_mean.drop("user_id",1,inplace=True)
df_teq_mean.drop('time',1,inplace=True)
df_teq_mean

,quality_0,quality_1,quality_2,quality_3,quality_4,quality_5,quality_6,quality_7,quality_8,quality_9,quality_10,quality_11,quality_12
0,0.000000,0.0,0.000000,0.0,0.0,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0
1,-0.750000,0.0,-0.750000,0.0,0.0,212.833328,-0.750000,0.0,0.0,0.0,2563.0,-0.750000,0.0
2,-1.250000,0.0,-1.250000,0.0,0.0,9.833333,-1.250000,0.0,0.0,0.0,133.0,-1.250000,0.0
3,0.000000,0.0,0.000000,0.0,0.0,1.000000,18.333333,220.0,0.0,0.0,12.0,0.000000,0.0
4,-0.500000,0.0,-0.500000,0.0,0.0,-0.333333,2.250000,33.0,0.0,0.0,2.0,-0.500000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,-0.916667,0.0,-0.916667,0.0,0.0,-0.750000,-0.916667,0.0,0.0,0.0,2.0,-0.916667,0.0
14995,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0
14996,-0.416667,0.0,-0.416667,0.0,0.0,2.666667,-0.416667,0.0,0.0,0.0,37.0,-0.416667,0.0
14997,0.000000,0.0,0.000000,0.0,0.0,1.083333,0.000000,0.0,0.0,0.0,13.0,0.000000,0.0


In [312]:
quality_test_error = np.hstack([np.asarray(df_teq_change),np.asarray(df_teq_count['time']).reshape(-1,1),np.asarray(df_teq_sum),np.asarray(df_teq_mean)])
quality_test_error

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         2.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  0.00000000e+00,  1.00000000e+00, ...,
         2.56300000e+03, -7.50000000e-01,  0.00000000e+00],
       [ 3.00000000e+00,  0.00000000e+00,  3.00000000e+00, ...,
         1.33000000e+02, -1.25000000e+00,  0.00000000e+00],
       ...,
       [ 2.00000000e+00,  0.00000000e+00,  2.00000000e+00, ...,
         3.70000000e+01, -4.16666667e-01,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         1.30000000e+01,  0.00000000e+00,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00]])

In [324]:
quality_test_error.shape

(14999, 40)

In [290]:
id_test_error = test_err[['user_id','errtype']].values
type_test_error = np.zeros((test_user_number,42))
for person_idx, err in tqdm(id_test_error):
    # person_idx - test_user_id_min 위치에 person_idx, errtype에 해당하는 error값을 +1
    type_test_error[person_idx - test_user_id_min,err - 1] += 1
type_test_error = type_test_error.reshape(type_test_error.shape[0],-1)
print(type_test_error.shape)

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:51<00:00, 323627.56it/s]

(14999, 42)


### 모델 테스트 데이터 생성

In [291]:
id_test_model_error = test_err_model[['user_id','model_nm']].values
model_test_error = np.zeros((test_user_number,9)) # 모델 카테고리 9개

for person_idx, err in tqdm(id_test_model_error):
    model_test_error[person_idx - test_user_id_min,err - 1] += 1
model_test_error.shape

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:51<00:00, 320762.22it/s]


(14999, 9)

### 펌웨어 테스트 데이터 생성

In [292]:
id_test_fwver_error = test_err_fwver[['user_id','fwver']].values
fwver_test_error = np.zeros((test_user_number,10)) # fwver 카테고리 10개

for person_idx, err in tqdm(id_test_fwver_error):
    fwver_test_error[person_idx - test_user_id_min,err - 1] += 1
fwver_test_error.shape

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:50<00:00, 327342.85it/s]


(14999, 10)

### 에러코드 테스트 데이터 생성

In [293]:
id_test_code_error = test_err_code[['user_id','errcode']].values
code_test_error = np.zeros((test_user_number,42)) # errcode 카테고리 42개

for person_idx, err in tqdm(id_test_code_error):
    code_test_error[person_idx - test_user_id_min,err - 1] += 1
code_test_error.shape

100%|██████████████████████████████████████████████████████████████████| 16532648/16532648 [00:50<00:00, 330423.50it/s]


(14999, 42)

In [322]:
test_x = np.hstack([type_test_error, model_test_error, fwver_test_error, code_test_error,quality_test_error])

In [323]:
test_x.shape

(14999, 143)

In [325]:
test = pd.DataFrame(data=test_x)

In [326]:
predictions = predict_model(final_model, data = test)

In [327]:
predictions

,0,1,2,3,4,5,6,7,8,9,...,135,136,137,138,139,140,141,142,Label,Score
0,0.0,0.0,0.0,0.0,62.0,1.0,1.0,0.0,0.0,0.0,...,0.166667,0.416667,5.0,0.0,0.0,2.0,0.000000,0.0,1.0,0.8886
1,0.0,0.0,3.0,98.0,3.0,4.0,4.0,0.0,0.0,0.0,...,212.833328,-0.750000,0.0,0.0,0.0,2563.0,-0.750000,0.0,0.0,0.7734
2,0.0,0.0,0.0,0.0,79.0,2.0,1.0,0.0,0.0,0.0,...,9.833333,-1.250000,0.0,0.0,0.0,133.0,-1.250000,0.0,1.0,0.5149
3,0.0,0.0,0.0,0.0,107.0,2.0,2.0,0.0,0.0,0.0,...,1.000000,18.333333,220.0,0.0,0.0,12.0,0.000000,0.0,1.0,0.8042
4,0.0,0.0,0.0,2.0,6.0,1.0,1.0,0.0,0.0,0.0,...,-0.333333,2.250000,33.0,0.0,0.0,2.0,-0.500000,0.0,1.0,0.7222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14994,0.0,0.0,0.0,2.0,12.0,5.0,5.0,0.0,0.0,0.0,...,-0.750000,-0.916667,0.0,0.0,0.0,2.0,-0.916667,0.0,0.0,0.7941
14995,0.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.7686
14996,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,2.666667,-0.416667,0.0,0.0,0.0,37.0,-0.416667,0.0,1.0,0.7191
14997,0.0,0.0,0.0,1.0,4.0,2.0,2.0,0.0,0.0,0.0,...,1.083333,0.000000,0.0,0.0,0.0,13.0,0.000000,0.0,1.0,0.7960


In [328]:
x = []
for i in range(len(predictions['Score'])):
    if predictions['Label'][i] =='1.0':
        x.append(predictions['Score'][i])
    else:
        x.append(1-predictions['Score'][i])

In [329]:
x

[0.8886,
 0.22660000000000002,
 0.5149,
 0.8042,
 0.7222,
 0.5889,
 0.08120000000000005,
 0.0534,
 0.706,
 0.27969999999999995,
 0.12260000000000004,
 0.1594,
 0.8855,
 0.10960000000000003,
 0.1279,
 0.2811,
 0.2902,
 0.39459999999999995,
 0.3348,
 0.3103,
 0.8466,
 0.4538,
 0.15590000000000004,
 0.13390000000000002,
 0.5183,
 0.29300000000000004,
 0.2801,
 0.15580000000000005,
 0.18179999999999996,
 0.08009999999999995,
 0.23360000000000003,
 0.5758,
 0.2157,
 0.22419999999999995,
 0.36629999999999996,
 0.24309999999999998,
 0.12939999999999996,
 0.34850000000000003,
 0.17300000000000004,
 0.057599999999999985,
 0.29710000000000003,
 0.15869999999999995,
 0.07069999999999999,
 0.08989999999999998,
 0.13790000000000002,
 0.38660000000000005,
 0.27880000000000005,
 0.14290000000000003,
 0.34440000000000004,
 0.10980000000000001,
 0.2573,
 0.2529,
 0.34640000000000004,
 0.13170000000000004,
 0.028200000000000003,
 0.21340000000000003,
 0.2963,
 0.35350000000000004,
 0.41269999999999996,


In [330]:
sample_submssion = pd.read_csv(PATH+'sample_submission.csv')

In [331]:
sample_submssion['problem'] = x

In [332]:
sample_submssion.to_csv("submission10_qual40_t42_m9_f10_e42_blended.csv", index = False)
sample_submssion

,user_id,problem
0,30000,0.8886
1,30001,0.2266
2,30002,0.5149
3,30003,0.8042
4,30004,0.7222
...,...,...
14994,44994,0.2059
14995,44995,0.2314
14996,44996,0.7191
14997,44997,0.7960
